In [74]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
import numpy as np
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.linear_model import Lasso
from warnings import simplefilter
from variable_importance.dgp import DataGenerator
from variable_importance.variable_importance_scoring import importance_score, cross_validation_scores
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


import numpy as np
import pandas as pd
import fastsparsegams

class FastSparseSklearn(BaseEstimator, TransformerMixin):
    def __init__(self, max_support_size, tol=1e-8, lambda_0=0.025, gamma=0):
        # self.data = data 
        # self.labels = labels
        # self.data = data.to_numpy() if not isinstance(data, np.ndarray) else data
        # self.labels = labels.to_numpy() if not isinstance(labels, np.ndarray) else labels
        # self.data = self.transform(data)
        # self.num_features = np.shape(data)[1]
        self.max_support_size = max_support_size
        # self.labels = self.labels[0].T
        self.tol = tol
        self.lambda_0 = lambda_0
        self.gamma = gamma
        
    def transform(self, data):
        data = data.to_numpy() if not isinstance(data, np.ndarray) else data
        data = data.astype(float)
        return data
    
    def fit(self, data, labels):
        data = self.transform(data)
        labels = self.transform(labels)
        self.model = fastsparsegams.fit(data, labels, penalty="L0", max_support_size=self.max_support_size, algorithm = "CDPSI")
        self.feature_importances_ = self.model.coeff(lambda_0=self.lambda_0, gamma=self.gamma).toarray()

    def predict(self, X):
        X = self.transform(X)
        return self.model.predict(X, lambda_0=self.lambda_0, gamma=self.gamma)
        
    def feature_importances(self):
        print(self.model)
        sz = len(self.model.lambda_0[0])
        lambda0 = self.model.lambda_0[0][sz-1]
        print(self.model.coeff(lambda_0=lambda0))
        best = np.where(np.abs(self.model.coeff(lambda_0=lambda0).toarray()) > self.tol)[0]
        print(best)
        importances = [0] * self.num_features
        for ind in best:
            importances[ind-1] = self.model.coeff(lambda_0=lambda0).toarray()[ind]
        return importances

In [75]:
param_grid = {
    "max_support_size": [5, 10, 15, 20, 25],
    "tol": [1e-9, 1e-8, 1e-7, 1e-6],
    "lambda_0": [0.001, 0.005, 0.01, 0.05, 0.1]
}

dgp = DataGenerator(num_cols=10, num_rows=20, num_important=3, num_interaction_terms=0, effects='linear', noise_scale=0.5)
dataset = dgp.generate_data()

X = dataset.drop(["target"], axis=1)
y = dataset["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [76]:
fs = FastSparseSklearn(20)

rscv = RandomizedSearchCV(fs, param_grid, scoring='r2', verbose=0, cv=3, n_iter=10, n_jobs=2)
rscv.fit(X_train, y_train)
best_model = rscv.best_estimator_

y_test_pred = best_model.predict(X_test)
r2_score(y_test, y_test_pred)
best_model.feature_importances_

<11x1 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Column format>

In [72]:
dgp = DataGenerator(num_cols=10, num_rows=20, num_important=3, num_interaction_terms=0, effects='linear', noise_scale=0.5)
dataset = dgp.generate_data()

X = dataset.drop(["target"], axis=1)
y = dataset["target"]

X = X.to_numpy()
y = y.to_numpy()

X = X.astype(float)
y = y.astype(float)
model = fastsparsegams.fit(X, y, penalty="L0", max_support_size=20, algorithm="CDPSI")
y_pred = model.predict(x=X, lambda_0=0.032715, gamma=0)
y_pred.shape


(20, 1)